# Urban Heat Islands

In this notebook we'll be exploring the urban heat island effect by looking at the impact on surface temperature of roof color. We'll be replicating the process described here: http://urbanspatialanalysis.com/urban-heat-islands-street-trees-in-philadelphia/ but using Python tools rather than ESRI.

**Extra packages:** To run this notebook, you'll need the PyViz tools and a library of *top of atmosphere* calculations from [`rio-toa`](https://github.com/mapbox/rio-toa): `pip install rio-toa`

To get all the requires packages run: `conda env create --file environment.yml`.

**Data sources:** This notebook uses Landsat data from Google Cloud Storage as well as some geographic data from OpenDataPhilly.

In [ ]:
import intake
import numpy as np
import pandas as pd
import xarray as xr

import geopandas as gpd
import cartopy.crs as ccrs
from geoviews.tile_sources import EsriImagery

import holoviews as hv
import hvplot.xarray
import hvplot.pandas

hv.extension('bokeh')

Just some extra info about Landsat data:

In [ ]:
band_info = pd.DataFrame([
        (1,  "Aerosol", " 0.43 - 0.45",    0.440,  "30",         "Coastal aerosol"),
        (2,  "Blue",    " 0.45 - 0.51",    0.480,  "30",         "Blue"),
        (3,  "Green",   " 0.53 - 0.59",    0.560,  "30",         "Green"),
        (4,  "Red",     " 0.64 - 0.67",    0.655,  "30",         "Red"),
        (5,  "NIR",     " 0.85 - 0.88",    0.865,  "30",         "Near Infrared (NIR)"),
        (6,  "SWIR1",   " 1.57 - 1.65",    1.610,  "30",         "Shortwave Infrared (SWIR) 1"),
        (7,  "SWIR2",   " 2.11 - 2.29",    2.200,  "30",         "Shortwave Infrared (SWIR) 2"),
        (8,  "Panc",    " 0.50 - 0.68",    0.590,  "15",         "Panchromatic"),
        (9,  "Cirrus",  " 1.36 - 1.38",    1.370,  "30",         "Cirrus"),
        (10, "TIRS1",   "10.60 - 11.19",   10.895, "100 * (30)", "Thermal Infrared (TIRS) 1"),
        (11, "TIRS2",   "11.50 - 12.51",   12.005, "100 * (30)", "Thermal Infrared (TIRS) 2")],
    columns=['Band', 'Name', 'Wavelength Range (µm)', 'Nominal Wavelength (µm)', 'Resolution (m)', 'Description']).set_index(["Band"])
band_info

## Loading data

For this example, we will be using landsat data stored on Google Cloud Storage. We can use a list or urls to fetch only the bands that we need for these analyses. We use jinja template notation in intake to pass parameters to the `urlpath`.

In [ ]:
cat = intake.open_catalog('catalog.yml')

Let's take a look at what the `google_landsat_band` looks like:

In [ ]:
with open('catalog.yml') as f:
    for line in f.readlines():
        print(line.rstrip())

The following might feel a bit arbitrary, but we have chosen the path and row corresponding to the area over Philadelphia using the [earth explorer](https://earthexplorer.usgs.gov/). We have also found the product id of the particular date of interest using the same tool. With these values in hand, we can access parts of each file on Google Cloud Storage. 

In [ ]:
path = 14
row = 32
product_id = 'LC08_L1TP_014032_20160727_20170222_01_T1'

The first step to using intake is to initialize the catalog entry with user parameters to create a `data source`. 

In [ ]:
data_source = cat.google_landsat_band(path=path, row=row, product_id=product_id)
data_source

From this `data source` we can get a lazily loaded xarray object using dask. To make sure that we can inspect what dask is up to, it can be helpful to create a dask client

### Optional: Create a Dask Client

We can use this dask client to inspect what dask is spending time on and when tasks are actually running.

In [ ]:
from dask.distributed import Client
client = Client()
client

### To Dask
The data from the bands themselves are not loaded in this cell (that's why it doesn't take very long). Only some file metadata and the coordinates are loaded. For this workflow we don't ever need to load all the data into memory or onto disk.

In [ ]:
ds = data_source.to_dask()
ds

## Sub-setting to area of interest

So far we haven't downloaded any band data. Since we know that we are interested in Philadelphia, we can just take a smaller square of data that covers the extents of the city. First we need to know the projection of the dataset:

In [ ]:
ds.crs

We'll convert that into a cartopy projection object:

In [ ]:
crs = ccrs.epsg(32618)

In order to determine what constitutes Philadelphia, we need some geometry to specify the bounds of the city. We can get a GeoJSON of neighborhood data from [OpenDataPhilly](https://www.opendataphilly.org/dataset/philadelphia-neighborhoods/resource/6c61f240-aafe-478e-b993-b75fd09a93d6).

In [ ]:
url = 'https://github.com/azavea/geo-data/raw/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson'
geoms = gpd.read_file(url)
geoms.head()

We can compute the bounds of each of these neighborhoods and then using min and max get a rectangle that encompasses all of Philly.

In [ ]:
bounds = geoms.geometry.bounds
bounds.describe()

In [ ]:
lower_left_corner_lat_lon = bounds.minx.min(), bounds.miny.min()
upper_right_corner_lat_lon = bounds.maxx.max(), bounds.maxy.max()

print(lower_left_corner_lat_lon, upper_right_corner_lat_lon)

Using the `crs` defined above, we can transform these lat lons into map coordinates. 

In [ ]:
ll_corner = crs.transform_point(*lower_left_corner_lat_lon, ccrs.PlateCarree())
ur_corner = crs.transform_point(*upper_right_corner_lat_lon, ccrs.PlateCarree())

print(ll_corner, ur_corner)

Then we can use those corners to slice the data. If the subset is empty along x or y, the ordering of the coordinates might not be what you anticipated. Try swapping the order of arguments in the slice. 

In [ ]:
subset = ds.sel(x=slice(ll_corner[0], ur_corner[0]), y=slice(ur_corner[1], ll_corner[1]))

We can persist this slice of the dataset in memory for easy use later.

In [ ]:
subset = subset.persist()
subset

To check that we got the right area, we can do a simple plot of one of the bands and overlay the neighborhoods on top of it. We'll use [hvplot](https://hvplot.pyviz.org) to quickly create a [holoviews](https://holoviews.org) object rendered in [bokeh](https://bokeh.org).

In [ ]:
band_plot = subset.sel(band=4).hvplot(datashade=True, crs=crs, cmap='gray', height=500, width=500)
hood_plot = geoms.hvplot(geo=True, alpha=.5, c='mapname')

band_plot * hood_plot

Now that we have looked at our map and ensured that the area covers the city, we will just take this chunk of the bands to use for further analyses:

## Calculate NDVI

We'll set up the calculation for NDVI but we won't yet do any computations - our bands are actually dask arrays, which allow for lazy computation.

In [ ]:
NDVI = (subset.sel(band=5) - subset.sel(band=4)) / (subset.sel(band=5) + subset.sel(band=4))
NDVI = NDVI.where(NDVI < np.inf)
NDVI

In order to visualize NDVI, the data will need to be loaded and the NDVI computed. We can expect this to take some non-trivial amount of time.

In [ ]:
NDVI.hvplot(x='x', y='y', crs=crs, datashade=True, cmap='viridis', height=500, width=500).opts(tools=[])

## Calculate land surface temperature

Given the NDVI calculated above, we can determine land surface temperature. For ease, we'll use some *top of atmosphere* calculations that have already been written up as Python functions as part of rasterio work in the `rio_toa` module. We'll also need to specify one more for transforming at satellite temperature (brightness temp) to land surface temperature. For these calculations we'll use both Thermal Infrared bands - 10 and 11.

In [ ]:
from rio_toa import brightness_temp, toa_utils

In [ ]:
def land_surface_temp(BT, w, NDVI):
    """Calculate land surface temperature of Landsat 8
    
    temp = BT/1 + w * (BT /p) * ln(e)
    
    BT = At Satellite temperature (brightness)
    w = wavelength of emitted radiance (μm)
    
    where p = h * c / s (1.439e-2 mK)
    
    h = Planck's constant (6.626e-34 Js)
    s = Boltzmann constant (1.38e-23 J/K)
    c = velocity of light (2.998e8 m/s)
    """
    h = 6.626e-34
    c = 2.998e8
    s = 1.38e-23
    
    p = (h * c / s) * 1e6
    
    Pv = (NDVI - NDVI.min() / NDVI.max() - NDVI.min())**2
    e = 0.004 * Pv + 0.986
    
    return BT / 1 + w * (BT / p) * np.log(e)

Now we'll set up a helper function to retrieve all the parameters from the metadata and general Landsat info table, and calculated the land surface temperature for band 10 and 11.

In [ ]:
def land_surface_temp_for_band(band, data, units='F'):
    # params from general Landsat info
    w = band_info.loc[band]['Nominal Wavelength (µm)']
    
    # params from specific Landsat data text file for these images
    ML = metadata[f'RADIANCE_MULT_BAND_{band}']
    AL = metadata[f'RADIANCE_ADD_BAND_{band}']
    K1 = metadata[f'K1_CONSTANT_BAND_{band}']
    K2 = metadata[f'K2_CONSTANT_BAND_{band}']
    
    at_satellite_temp = brightness_temp.brightness_temp(data.sel(band=band).values, ML, AL, K1, K2)
    
    temp = land_surface_temp(at_satellite_temp, w, NDVI).compute()
    return toa_utils.temp_rescale(temp, units)

### Loading Metadata

Every Landsat product has an associated metadata file containing parameters for these particular Landsat images. We'll write a helper function to get these parameters from the matlab.txt file.

In [ ]:
def load_google_landsat_metadata(path, row, product_id):
    """Load Landsat metadata for path, row, product_id from Google Cloud Storage
    """
    def parse_type(x):
        try: 
            return eval(x)
        except:
            return x
    
    baseurl = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01'
    suffix = f'{path:03d}/{row:03d}/{product_id}/{product_id}_MTL.txt'
    
    df = intake.open_csv(
        urlpath=f'{baseurl}/{suffix}',
        csv_kwargs={'sep': '=',
                    'header': None,
                    'names': ['index', 'value'],
                    'skiprows': 2,
                    'converters': {'index': (lambda x: x.strip()),
                                   'value': parse_type}}).read()
    metadata = df.set_index('index')['value']
    return metadata

In [ ]:
metadata = load_google_landsat_metadata(path, row, product_id)
metadata.head()

> **To Do:** Might be easier to read from the json as in the [dask geotiff example](https://examples.dask.org/applications/satellite-imagery-geotiff.html).

We can use the functions to compute the land surface temperature for band 10 and band 11 - we'll combine these into on xarray object for later use.

In [ ]:
temp_10_f = land_surface_temp_for_band(10, subset)
temp_11_f = land_surface_temp_for_band(11, subset)

temp_f = xr.concat([temp_10_f, temp_11_f], 
                   dim=xr.DataArray([10,11], name='band', dims=['band']))
temp_f

Compare the results from the two different bands. Band 10 gets a higher temp over all, but the patterns of where the high temps occur seem similar. 

In [ ]:
temp_f.hvplot(x='x', y='y', groupby='band', cmap='fire_r', 
              crs=crs, rasterize=True, height=450, width=400, 
              colorbar=False).layout()

We'll take the mean of the calculated land surface temperature for each of the bands and mimic the colormap used in the project that we are duplicating.

In [ ]:
mean_temp_f = temp_f.mean(dim='band')

p = mean_temp_f.hvplot(x='x', y='y', title='Mean Surface Temp (F)', crs=crs, 
                       height=500, width=550, cmap='rainbow', alpha=.5, legend=False)
p * EsriImagery

Notice how the hot spots are located over warehouse roofs and parking lots. This becomes even more visible when just the temperatures greater than 90F are displayed. To show this, we'll make a special colormap that just includes high intensity reds that are found at the top of the `fire_r` colormap.

In [ ]:
import colorcet as cc

special_cmap = cc.fire[::-1][90:]

In [ ]:
thresholded_temp_p = (mean_temp_f.where(mean_temp_f > 90)
    .hvplot(x='x', y='y', title='Mean Temp (F) > 90',
            cmap=special_cmap, crs=crs, height=500, width=450, 
            colorbar=False, legend=False)
    .redim(value='Temperature (F)'))

thresholded_temp_p + thresholded_temp_p.options(alpha=.3) * EsriImagery

Now what if we want the user to be able to set their own threshold and update the plot? Or what if we want to add in the street tree data?

### Next Notebook
Since this notebook is getting unwieldy we can save off the mean temperature as an intermediary product. 

In [ ]:
mean_temp_f.name = 'temperature'
mtf_ds = mean_temp_f.to_dataset()

mtf_ds.attrs['crs'] = subset.attrs['crs']
mtf_ds.attrs['units'] = 'Fahrenheit'
mtf_ds

The following cell will try to save data into the path. If data already exists in that location, it will fail.

Now we have saved our work and can move on to [the next notebook](02_trees.ipynb) to continue the analysis.